In [ ]:
import cv2

In [ ]:
# Classifiers

eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
# Import glasses img
glasses = cv2.imread('imageedit_1_9641209111.png', -1)

# Create the mask for the glasses
glasses_gray = cv2.cvtColor(glasses, cv2.COLOR_BGR2GRAY)
ret, orig_mask = cv2.threshold(glasses_gray, 10, 255, cv2.THRESH_BINARY)

In [ ]:
# Create mask
orig_mask_inv = cv2.bitwise_not(orig_mask)

In [ ]:
glasses = glasses[:,:,0:3]
orig_height, orig_width = glasses.shape[:2]

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    # Capture video feed
    ret, frame = cap.read()
    
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
 
    # Create greyscale image from the video feed
    gray = cv2.cvtColor(small_frame, cv2.COLOR_BGR2GRAY)
 
    # Detect faces in input video stream
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
 
   # Iterate over each face found
    for (x, y, w, h) in faces:
        face = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
 
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
 
        # Detect eyes within the region bounded by each face (the ROI)
        eyes = eye_cascade.detectMultiScale(roi_gray)
 
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255,0,0),2)
 
            # The glasses should be three times the width of the eyes
            glasses_width =  3 * ew
            glasses_height = glasses_width * orig_height / orig_width
 
            # Center the mustache on the bottom of the nose
            x1 = ex - (glasses_width / 4)
            x2 = ex + ew + (glasses_width / 4)
            y1 = ey + eh - (glasses_height / 2)
            y2 = ey + eh + (glasses_height / 2)
 
            # Check for clipping
            if x1 < 0:
                x1 = 0
            if y1 < 0:
                y1 = 0
            if x2 > w:
                x2 = w
            if y2 > h:
                y2 = h
 
            # Re-calculate the width and height of the mustache image
            glasses_width = x2 - x1
            glasses_height = y2 - y1
 
            # Re-size the original image and the masks to the mustache sizes
            # calcualted above
            dsize = (int(glasses_width),int(glasses_height))
            
            glasses = cv2.resize(glasses, dsize, interpolation = cv2.INTER_AREA)
            mask = cv2.resize(orig_mask, dsize, interpolation = cv2.INTER_AREA)
            mask_inv = cv2.resize(orig_mask_inv, dsize, interpolation = cv2.INTER_AREA)
 
            # take ROI for mustache from background equal to size of mustache image
            roi = roi_color[int(y1):int(y2), int(x1):int(x2)]
 
            # roi_bg contains the original image only where the mustache is not
            # in the region that is the size of the mustache.
            roi_bg = cv2.bitwise_and(roi, roi, mask = mask_inv)
 
            # roi_fg contains the image of the mustache only where the mustache is
            roi_fg = cv2.bitwise_and(glasses, glasses, mask = mask)
 
            # join the roi_bg and roi_fg
            dst = cv2.add(roi_bg,roi_fg)
 
            # place the joined image, saved to dst back over the original image
            roi_color[int(y1):int(y2), int(x1):int(x2)] = dst
 
            break
 
    # Display the resulting frame
    cv2.imshow('Video', frame)
 
    # press any key to exit
    # NOTE;  x86 systems may need to remove: " 0xFF == ord('q')"
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break